In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
azure_search_endpoint = os.getenv('AZURE_SEARCH_ENDPOINT')
azure_search_key = os.getenv('AZURE_SEARCH_KEY')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
openai_deployment = os.getenv('AZURE_OPENAI_DEPLOYMENT')
openai_api_version = os.getenv('OPENAI_API_VERSION')


In [2]:
azure_search_endpoint

'https://vector-storage.search.windows.net'

In [2]:
with open('anatomy.txt', 'r', encoding='utf-8') as f:
          file = f.read()

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
          chunk_size = 600,
          chunk_overlap = 0,
          length_function = len
)

chunks = text_splitter.split_text(file)

In [5]:
from langchain_community.vectorstores import AzureSearch
from langchain_ollama import OllamaEmbeddings
from langchain.schema import Document

documentos = [Document(page_content=chunk) for chunk in chunks]

chunk_store = AzureSearch.from_documents(
          azure_search_endpoint = azure_search_endpoint,
          azure_search_key = azure_search_key,
          documents = documentos,
          embedding=OllamaEmbeddings(model = 'nomic-embed-text:latest')
)

In [20]:
chunks_vector = chunk_store.as_retriever()

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import AzureChatOpenAI

class GradeDocuments(BaseModel):
          binary_score : str = Field(
                  description="Doments are relevant to the question 'yes' or 'no'"
          )

llm = AzureChatOpenAI(
        azure_deployment=openai_deployment,
        azure_endpoint= azure_openai_endpoint,
        api_key= openai_api_key,
        api_version= openai_api_version,
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

In [22]:
# Prompt
system = """Eres un calificador que evalúa la relevancia de un documento recuperado con respecto a una pregunta del usuario. \n 
    No tiene por qué ser una prueba rigurosa. El objetivo es filtrar las recuperaciones erróneas. \n
    Si el documento contiene palabras clave o significados semánticos relacionados con la pregunta del usuario, califíquelo como pertinent. \n
    Dar una puntuación binaria «sí» o «no» para indicar si el documento es pertinente para la pregunta."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "agent memory"
docs = chunks_vector.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

C:\Users\argui\AppData\Local\Temp\ipykernel_19236\2356134980.py:15: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = chunks_vector.get_relevant_documents(question)


binary_score='no'


In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)



rag_chain = prompt | llm | StrOutputParser()

generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)